In [1]:
%reload_ext watermark
%reload_ext autoreload
%autoreload 2
%watermark -v -p numpy,sklearn,pandas
%watermark -v -p cv2,PIL,matplotlib
%watermark -v -p torch,torchvision,torchaudio,pytorch_lightning
%matplotlib inline
%config InlineBackend.figure_format='retina'
%config IPCompleter.use_jedi = False

from IPython.display import display, HTML, Javascript
display(HTML('<style>.container { width:%d%% !important; }</style>' % 90))

def _IMPORT_(x):
    try:
        exec(x, globals())
    except:
        pass


CPython 3.6.9
IPython 7.16.1

numpy 1.19.4
sklearn 0.24.0
pandas 1.1.5
CPython 3.6.9
IPython 7.16.1

cv2 4.5.1
PIL 6.2.2
matplotlib 3.3.3
CPython 3.6.9
IPython 7.16.1

torch 1.8.0.dev20210103+cu101
torchvision 0.9.0.dev20210103+cu101
torchaudio not installed
pytorch_lightning 1.2.0


In [2]:
###
### Common ###
###

import os, io, time, random, math, base64
import json, requests, sys
import os.path as osp

_IMPORT_('import numpy as np')
_IMPORT_('import pandas as pd')
_IMPORT_('from tqdm.notebook import tqdm')

def print_progress_bar(x):
    print('\r', end='')
    print('Progress: {}%:'.format(x), '%s%s' % ('▋'*(x//2), '.'*((100-x)//2)), end='')
    sys.stdout.flush()

###
### Torch ###
###

_IMPORT_('import torch')
_IMPORT_('import torch.nn as nn')
_IMPORT_('import torch.nn.functional as F')
_IMPORT_('import torch.optim as O')
_IMPORT_('from torchvision import models as M')
_IMPORT_('from torchvision import transforms as T')
_IMPORT_('from torch.utils.data import Dataset, DataLoader')

###
### Display ###
###

_IMPORT_('import cv2')
_IMPORT_('from PIL import Image')
_IMPORT_('from torchvision.utils import make_grid')
_IMPORT_('import matplotlib.pyplot as plt')
_IMPORT_('import plotly')
_IMPORT_('import plotly.graph_objects as go')

# plotly.offline.init_notebook_mode(connected=False)

def show_video(video_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    mp4 = open(video_path, 'rb').read()
    data_url = 'data:video/mp4;base64,' + base64.b64encode(mp4).decode()
    return HTML('<video %s %s controls src="%s" type="video/mp4"/>' % (W, H, data_url))

def show_image(image_path, width=None, height=None):
    W, H = '', ''
    if width:
        W = 'width=%d' % width
    if height:
        H = 'height=%d' % height
    img = open(image_path, 'rb').read()
    data_url = 'data:image/jpg;base64,' + base64.b64encode(img).decode()
    return HTML('<img %s %s src="%s"/>' % (W, H, data_url))

from k12libs.utils.nb_easy import RACEURL

In [3]:
API_INFERENCE = f'{RACEURL}/raceai/framework/inference'
API_POPMSG = f'{RACEURL}/raceai/private/popmsg'
MODEL_TASK = 'zmq.sense.inference'
MSGKEY = 'zmq.sense.test'
TEST_SAMPLES = {
    'test1': 'https://raceai.s3.didiyunapi.com/data/media/videos/sense/handclap.mp4',
    'test2': 'https://raceai.s3.didiyunapi.com/data/media/videos/sense/thumb_up.mp4',
}
API_INFERENCE, API_POPMSG 

('http://116.85.5.40:9119/raceai/framework/inference',
 'http://116.85.5.40:9119/raceai/private/popmsg')

## 接口测试

In [9]:
def get_result():
    while True:
        result = json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
        if len(result) > 0 and 'progress' in result[-1]:
            progress = int(result[-1]['progress'])
            print_progress_bar(progress)
            if progress >= 100.0:
                # print('\nOutput:\n\n', json.dumps(result[-1], indent=4))
                return result[-1]
        time.sleep(1)

### 输入

```json
{
    "task": "zmq.sense.inference",  // zmq的任务类型, 不用修改
    "cfg": {
        "pigeon": { // 信鸽数据, key的内容由后端自行添加, 其中必须含有msgkey和user_code, 可以任意添加其他key, 返回时会携带pigeon
            "msgkey": "zmq.sense.test",  // 任务过程产生的数据, 会发送到msgkey的redis队列中
            "user_code": "123", // 这个任务属于哪个用户
        },
        "video": `path` // 视频地址
        "save_video": False, // 是否保存处理后的视频
    }
}
```

In [6]:
_TEMPLATE_ = '''{
    "task": "%s",
    "cfg": {
        "pigeon": {
            "msgkey": "%s",
            "user_code": "%s",
        },
        "video": "%s",
        "save_video": False,
    }
}'''

cfgdatas = [ _TEMPLATE_ % (
    MODEL_TASK, MSGKEY,
    user_code, video_url
) for user_code, video_url in TEST_SAMPLES.items()]

print("Input:\n\n", cfgdatas[-1])

Input:

 {
    "task": "zmq.sense.inference",
    "cfg": {
        "pigeon": {
            "msgkey": "zmq.sense.test",
            "user_code": "test2",
        },
        "video": "https://raceai.s3.didiyunapi.com/data/media/videos/sense/thumb_up.mp4",
        "save_video": False,
    }
}


### 输出

接口进度返回:

```json
{
    "pigeon": {  // 信鸽数据, 直接透传回去
        "msgkey": "zmq.sense.test",
        "user_code": "123"
    },
    "task": "zmq.sense.inference",
    "errno": 0,  // 当errno < 0 时, 表示有错误发生
    "progress": 100.0,  // 任务的进度
    "target_json": "https://raceai.s3.didiyunapi.com/outputs/123/sense-results.json", // 只有进度为100.0时, 返回关键点结果json数据
    "target_mp4": "https://raceai.s3.didiyunapi.com/outputs/123/sense-target.mp4" // 只有进度为100.0时, 返回带有关键点标注的mp4数据
}
```

最终结果内容:

```json

{
    "frame_width": 256,        // (期望)视频帧宽度
    "frame_height": 256,       // (期望)视频帧高度
    "model_step": 4,           // 模型帧步长
    "fps": 16,                 // 模型帧率
    "num_frames": 79,          // 模型帧率下的帧总数
    "action_recognition": [    // 动作识别, 每一帧的信息
        
         ...
        
        {
            "image_id": "47.jpg",       // 第几帧
            "at_time": 3.0,             // 当前帧对应的时间
            "sorted_predictions": {     // 动作预测结果(排序后)
                "C1": 0.981,            // 动作"C1"的分数
                "C2": 0.019
            }
        },
        {
            "image_id": "48.jpg",
            "at_time": 3.062,
            "sorted_predictions": {
                "C1": 0.976,
                "C2": 0.024
            }
        },

        ...
        
    ]
}

```

In [12]:
reqdata = eval(cfgdatas[-1])
json.loads(requests.get(url=f'{API_POPMSG}?key={MSGKEY}').text)
json.loads(requests.post(url=API_INFERENCE, json=reqdata).text)
print(reqdata)
result = get_result()
result

{'task': 'zmq.sense.inference', 'cfg': {'pigeon': {'msgkey': 'zmq.sense.test', 'user_code': 'test2'}, 'video': 'https://raceai.s3.didiyunapi.com/data/media/videos/sense/thumb_up.mp4', 'save_video': False}}
Progress: 100%: ▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋▋

{'pigeon': {'msgkey': 'zmq.sense.test', 'user_code': 'test2'},
 'task': 'zmq.sense.inference',
 'errno': 0,
 'progress': 100.0,
 'target_json': 'https://raceai.s3.didiyunapi.com/outputs/test2/sense-results.json',
 'target_mp4': 'https://raceai.s3.didiyunapi.com/outputs/test2/sense-target.mp4'}

In [13]:
json_result = requests.get(result['target_json']).json()
print(json.dumps(json_result, indent=4))

{
    "frame_width": 256,
    "frame_height": 256,
    "model_step": 4,
    "fps": 16,
    "num_frames": 79,
    "action_recognition": [
        {
            "image_id": "0.jpg",
            "at_time": 0.062,
            "sorted_predictions": {
                "C2": 0.0,
                "C1": 0.0
            }
        },
        {
            "image_id": "1.jpg",
            "at_time": 0.125,
            "sorted_predictions": {
                "C2": 0.0,
                "C1": 0.0
            }
        },
        {
            "image_id": "2.jpg",
            "at_time": 0.188,
            "sorted_predictions": {
                "C2": 0.0,
                "C1": 0.0
            }
        },
        {
            "image_id": "3.jpg",
            "at_time": 0.25,
            "sorted_predictions": {
                "C2": 0.0,
                "C1": 0.0
            }
        },
        {
            "image_id": "4.jpg",
            "at_time": 0.312,
            "sorted_predictions": {
       

## References